In [1]:
# 4.1 การส่งสัญญาณฐานสองแบบตั้งฉาก
# probbility of error for
# orthogonal binary signal transmission
# using simulation method
# define Q-function
# using the complementary error function
function Q(x)
	return 1/2*erfc.(x/sqrt(2))
end;

# function for simulation
function smpe(SNR)
	E=1 # fixed energy of E=1
	sgma=sqrt(E/2/SNR) #stdev of n0, n1
	# generate binary data source 10000 bits
	N=10000
	dsource=Array{Int8}(zeros(N))
	for i=1:N
		if rand()<0.5
			dsource[i]=0
		else
			dsource[i]=1
		end
	end
	# detection and compute probability of error
	numoferror=0
	for i=1:N
		#correlator outputs
		if dsource[i]==0
			r0=E+sgma*randn()
			r1=sgma*randn()
		else
			r0=sgma*randn()
			r1=E+sgma*randn()
	end
		# detection
		if (r0>r1)
			decis=0
		else
			decis=1
		end
		if decis != dsource[i]
		numoferror+=1
		end
	end
	p=numoferror/N
	return p
end;

In [2]:
# main
using Plots
using LaTeXStrings
plotly()
using SpecialFunctions

In [3]:
# theoretical
SNR_theo=range(1,stop=10^1.1,length=100)
SNR_theo_db=10*log10.(SNR_theo)
Pe_theo=Q(sqrt.(SNR_theo))

100-element Vector{Float64}:
 0.15865525393145707
 0.14527585435763385
 0.13330303017609585
 0.12253513558316836
 0.11281079102774
 0.10399832236218937
 0.09598857228931515
 0.0886898431617829
 0.08202423850991142
 0.07592495236231739
 0.07033421875769552
 0.0652017322954931
 0.060483411939296905
 ⋮
 0.00038721014340724433
 0.00036355794937302293
 0.0003413644827250913
 0.0003205386943235792
 0.0003009953043169477
 0.0002826544295027433
 0.00026544123535448416
 0.00024928561102986063
 0.0002341218657952146
 0.00021988844541207523
 0.00020652766713411257
 0.0001939854720578607

In [4]:
# simulation
SNR_sim_db=collect(1:11)
SNR_sim=10 .^(SNR_sim_db/10) # need space before dot
Pe_sim=zeros(length(SNR_sim_db))
for i=1:length(SNR_sim_db)
	SNR=SNR_sim[i]
	Pe_sim[i]=smpe(SNR)
end
#semilogy(x,y)
# theoretical plot
plot(SNR_theo_db,Pe_theo,legend=:none,yscale=:log10)
plot!(xlabel="10log₁₀ E/N0 (dB)",ylabel="Pₑ")
# simulation scatter
scatter!(SNR_sim_db,Pe_sim,yscale=:log10)

In [5]:
# 4.2 การส่งสัญญาณแบบ 4 ระดับแอมพลิจูด
# define Q-function
# using the complementary error function
function Q_2(x)
    return 1/2*erfc(x/sqrt(2))
end;

In [7]:
# function for simulating 4PAM
function smldPe(snr_in_dB)
	d=1
	SNR=10^(snr_in_dB/10)
	sgma=sqrt( (5*d^2)/(4*SNR) )
	N=10000
	# generation of the quarternary data source follows
	dsource=Array{Int8}(zeros(N))
	for i=1:N
		temp=rand()
		if (temp<0.25)
			dsource[i]=0
		else
			if (temp<0.5)
				dsource[i]=1
			else
				if (temp<0.75)
					dsource[i]=2
				else
					dsource[i]=3
				end
			end
		end
	end
	# detection and probability of error
	numoferr=0
	for i=1:N
		# correlator output
		if (dsource[i]==0)
			r = -3d+sgma*randn() # if source output is "00"
		else
			if (dsource[i]==1)
				r = -d+sgma*randn() #if source output is "01"
			else
				if (dsource[i]==2)
				r = d+sgma*randn() #if source output is "10"
				else
				r = 3d+sgma*randn() #if source output is "11"
				end
			end
		end
		# detector follows
		if (r< -2d)
			decis=0 # decision is "00"
		else
			if (r<0)
				decis=1 # decision is "01"
			else
				if (r<2d)
					decis=2 # decision is "10"
				else
					decis=3 # decision is "11"
				end
			end
		end
		if (decis!=dsource[i]) # if error increment counter
			numoferr=numoferr+1
		end
	end
	p=numoferr/N #prob of error estimation
	return p
end;

In [8]:
# main for simulation 4PAM
using Plots
using LaTeXStrings
plotly()
using SpecialFunctions #this packet is for erfc()

In [ ]:
SNRindB1=collect(0:12)
SNRindB2=range(0,stop=12,length=100)
smld_err_prob=zeros(length(SNRindB1))
for i=1:length(SNRindB1)
	#simulation error rate
	smld_err_prob[i]=smldPe(SNRindB1[i])
end

theo_err_prob=zeros(length(SNRindB2))
for i=1:length(SNRindB2)
	#signal to noise ratio
	SNR_per_bit=10^(SNRindB2[i]/10)
	# theoretical error rule
	theo_err_prob[i]=3/2*Q_2(sqrt(4/5*SNR_per_bit))
end

#semilogy(x,y)
# theoretical plot
plot(SNRindB2,theo_err_prob,label="Theoretical",yscale=:log10)
plot!(xlabel="10log₁₀ E/N0(dB)",ylabel="P\_e<tab>")
#xlabel mean 10log10 Eavb/N0
# simulation scatter
scatter!(SNRindB1,smld_err_prob,label="Simulation",yscale=:log10)